In [0]:
tiers = ["bronze", "silver", "gold"]
adls_paths = {tier : f"abfss://{tier}@agristorage2025.dfs.core.windows.net" for tier in tiers}

#Accessing path
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)


[]

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
# Pour tester l'API
# today = date.today().strftime("%Y-%m-%d")
# # les coordonnées de localisation (ex. Paris)
# latitude = 48.85
# longitude = 2.35

In [0]:
dbutils.widgets.text("latitude", "")
latitude = dbutils.widgets.get("latitude")
dbutils.widgets.text("longitude", "")
longitude = dbutils.widgets.get("longitude")
dbutils.widgets.text("today", "")
today = dbutils.widgets.get("today")
dbutils.widgets.text("ville", "")
ville = dbutils.widgets.get("ville")
dbutils.widgets.text("pays", "")
pays = dbutils.widgets.get("pays")

In [0]:
print("Latitude =", latitude)
print("Longitude =", longitude)
print("Date =", today)
print("ville =", ville)
print("pays =", pays)

In [0]:
# Construire l'URL de l'API Open-Meteo
url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m,soil_temperature_0cm,precipitation&timezone=Europe%2FParis"

try:
    # Envoyer la requête GET pour récupérer les données
    response = requests.get(url)

    # Vérifier que la requête a réussi
    response.raise_for_status()  # Lève une erreur HTTP si la requête échoue

    # Extraire les données JSON
    data = response.json().get("hourly", {})

    # Vérifier si les données existent
    if not data:
        print("No data returned for the specified location.")
    else:
        data_length = len(data.get("time", []))
        data["latitude"] = [latitude] * data_length
        data["longitude"] = [longitude] * data_length
        
        # Spécifier le chemin ADLS (Azure Data Lake Storage)
        # file_path = f"{bronze_adls}/openmeteo_{today}.json"
        file_path = f"{bronze_adls}/meteo/{pays}_{ville}_{today}.json"

        # 💾 Convertir en JSON formaté
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

Wrote 16575 bytes.
Data successfully saved to abfss://bronze@agristorage2025.dfs.core.windows.net/openmeteo_2025-04-13.json


In [0]:
output_data = {
    "latitude": latitude,
    "longitude": longitude,
    "today": today,
    "ville": ville,
    "pays": pays,
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

output_json = json.dumps(output_data)
print(f"Serialized JSON: {output_json}")
dbutils.notebook.exit(output_json)